In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
df = pd.read_csv('../input/titanic/train.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


# Initial thoughts on each field:
* 1) PassengerId  891 non-null    int64  - Meaningless key: delete
* 2) Survived     891 non-null    int64  - The Boolean target to predict
* 3) Pclass       891 non-null    int64  - 1, 2, 3. Proxy for social class, so use as an integer
* 4) Name         891 non-null    object - Scope for feature engineering a person's title, also Swedish (lots of passengers) names often end in _sen
* 5) Sex          891 non-null    object - Simple male/female. No non-binary in 1912!
* 6) Age          714 non-null    float64- 80% present. Worth imputing the missing 20%
* 7) SibSp        891 non-null    int64  - Explore whether families all die/survive together
* 8) Parch        891 non-null    int64  - Explore whether families all die/survive together
* 9) Ticket       891 non-null    object - I can't imagine the ticket code has any importance: delete
* 10) Fare         891 non-null    float64- Originally LSD, but converted to decimal. Multiples of guineas (£1.05) might show upper class
* 11) Cabin        204 non-null    object - 23% present. First letter probably shows the deck, ie proximity to lifeboat
* 12) Embarked     889 non-null    object - Possible discrimination against Irish passengers

In [8]:
print(df['Survived'].sum()/891*100, '% survival rate')

38.38383838383838 % survival rate


# 3) Passenger Class
1, 2 or 3. 
Proxy for social class, so use it as an integer, not categorical Booleans

In [9]:
df['Pclass'].unique()

array([3, 1, 2])

In [10]:
passengers_by_class = df.groupby('Pclass').count()['PassengerId']

In [11]:
passengers_by_class

Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64

In [12]:
survival_by_class = df.groupby('Pclass').sum()['Survived']

In [13]:
survival_by_class

Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64

In [14]:
survival_by_class['total']=passengers_by_class

In [15]:
survival_by_class

Pclass
1                                                      136
2                                                       87
3                                                      119
total    Pclass
1    216
2    184
3    491
Name: Passen...
Name: Survived, dtype: object

In [16]:
survival_rate_by_class = np.divide(survival_by_class, passengers_by_class)

In [17]:
survival_rate_by_class

Pclass
1                                                 0.629630
2                                                 0.472826
3                                                 0.242363
total    Pclass
1   NaN
2   NaN
3   NaN
Name: Passenger...
dtype: object

## No surprise: negative correlation between class (number) and survival

# 4) Name

Scope for feature engineering a person's title. <br>
I read somewhere that there were lots of Swedish passengers, whose names often end in _sen <br>
Format: Surname, Title Forenames eg Tart, Mr Nigel

In [18]:
#df[['surame', 'title_forenames']] = 

pd.series(df['Name']).str.split(',')

AttributeError: module 'pandas' has no attribute 'series'

In [ ]:
df['Name'].str.split(',')[1]

# 5) Sex

Simple male/female. No non-binary in 1912!

# 6) Age

80% present. Worth imputing the missing 20%

# 7) Sibling/Spouse

Explore whether families all die/survive together

# 8) Parent/Child

Explore whether families all die/survive together

# 9) Ticket Code

I can't imagine the ticket code has any importance: delete

# 10) Fare

Originally LSD, but converted to decimal. Multiples of guineas (£1.05) might show upper class

# 11) Cabin Number

Only 23% present, so probably drop the field, but the first letter probably shows the deck, ie proximity to lifeboat

# 12) Place of Embarkation

Possible discrimination against Irish passengers

In [ ]:
X_train = df.drop('Survived', axis=1)
y_train = df['Survived']